# Subset scenes

In [62]:
import os
import matplotlib.pyplot as plt
import numpy as np
import shapely
import geopandas as gpd
import rasterio as rio
from rasterio.windows import Window
import rasterio.mask 
import shutil

In [67]:
# load, crop, and save rasters

# list interferograms
dirs = os.listdir('/Users/qbren/Desktop/uw_courses/2022_spring/inferring-structure/project/data_processing/data_raw_test')

# loop through interferograms
for i, idir in enumerate(dirs):
    cwd = f'/Users/qbren/Desktop/uw_courses/2022_spring/inferring-structure/project/data_processing/data_raw_test/{idir}'
    os.chdir(cwd)
    
    # select the line of sight displacement
    allfiles = os.listdir(cwd)
    for fn in allfiles:
        if fn[-12:] == 'los_disp.tif':
            tif = fn
    tif_fn = f'/Users/qbren/Desktop/uw_courses/2022_spring/inferring-structure/project/data_processing/data_raw_test/{idir}/{tif}'
     
    xsize, ysize = 125, 125  # define the test image
    print("working on: ", tif)
    subset_number = 20  # define the number of subsets
    subsets = 0
    
    # loop until subsetting is finished
    while subsets < subset_number:
        
        # open src image
        src = rio.open(tif_fn)
        rs = src.read(1) # also open as np array
        
        # create bounds for new origin location
        xmin, xmax = 0, src.width - xsize
        ymin, ymax = 0, src.height - ysize
        xoff, yoff = np.random.randint(xmin, xmax), np.random.randint(ymin, ymax)
        
        # only save if there are no nodata values in subset 
        if not 0.0 in rs[yoff:(yoff+ysize), xoff:(xoff+xsize)]:
            window = Window(xoff, yoff, xsize, ysize)
            transform = src.window_transform(window)

            # write subset
            profile = src.profile
            profile.update({
                'height': xsize,
                'width': ysize,
                'transform': transform})
            subsets+=1 # update index
        
            out_fn = f'/Users/qbren/Desktop/uw_courses/2022_spring/inferring-structure/project/data_processing/data_crop/sub{subsets}nocorr_{tif}'

            with rasterio.open(out_fn, 'w', **profile) as dst:
                # Read the data from the window and write it to the output raster
                dst.write(src.read(window=window))
            src.close()

working on:  S1AA_20190505T135154_20190622T135157_VVP048_INT80_G_weF_011A_los_disp.tif
